In [ ]:
# Last amended: 2nd August, 2021
# Myfolder: C:\Users\Administrator\OneDrive\Documents\talkingdata
# Ref: https://www.kaggle.com/nanomathias/feature-engineering-importance-testing
#      https://www.kaggle.com/nuhsikander/lgbm-new-features-corrected
#      https://github.com/harnalashok/h2o
#
# Data source: https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection
# Objectives:
#           i)    Categorical feature engineering
#                 using aggregation

In [ ]:
# Mount your google drive 
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# 1.1 Call libraries
# %reset -f
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random, gc
import os,time

In [ ]:
# 1.2 Display output from multiple commands
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# 2.0 Import warnings module
import warnings
# 2.1 Do not print warnings on screen
warnings.filterwarnings("ignore")

In [ ]:
# 1.3 Change folder to where TalkingData is:
#os.chdir("C:\\Users\\Administrator\\OneDrive\\Documents\\talkingdata")
path =  "/content/drive/MyDrive/loan-default-prediction/"
os.listdir(path)

['sampleSubmission.csv', 'test_v2.csv.zip', 'train_v2.csv.zip']

In [ ]:
Loan_Data = pd.read_csv(path + "train_v2.csv.zip")
Loan_Data.head()
Loan_Data_Test=pd.read_csv(path + "test_v2.csv.zip")
Loan_Data_Test.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,...,f740,f741,f742,f743,f744,f745,f746,f747,f748,f749,f750,f751,f752,f753,f754,f755,f756,f757,f758,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768,f769,f770,f771,f772,f773,f774,f775,f776,f777,f778,loss
0,1,126,10,0.686842,1100,3,13699,7201.0,4949.0,126.75,126.03,7,0.7607,0.7542,612922,0.7236,0.7236,0.5171,0.7236,0.8476,0.7876,1.097851e+09,89,66,998046.0,89.0,89.0,89.00,89.00,89.0,0.9271,0,0,0,5,0,0,0.73180,0.72124,0.749280,...,-4.40,6,9.76,-21.82,49.18,1.0435,-0.9066,1.5883,3.4154,2.8283,2.3451,0.030594,1.7418,1.5271,0.8474,0.4715,0.028362,3.1611,2.5162,2.0037,0.019636,4.4352,4.2676,-0.1524,1,-0.40,-0.560,-0.440,-0.6280,-3.14,5,2.14,-1.54,1.18,0.1833,0.7873,1,0,5,0
1,2,121,10,0.782776,1100,3,84645,240.0,1625.0,123.52,121.35,7,0.6555,0.6555,245815,0.6341,0.6341,0.3909,0.6667,0.6903,0.6903,8.449459e+08,78,50,754416.0,78.0,78.0,78.00,78.00,78.0,0.8478,0,0,0,6,0,0,0.65705,0.64069,0.626200,...,0.35,7,51.73,-72.44,798.74,2.7180,-0.8294,1.0740,2.9194,2.1180,1.5666,0.120442,1.1963,1.0322,0.4843,0.2389,0.130160,2.7659,1.9523,1.4059,0.115277,3.2763,2.7962,-0.3097,1,-0.17,-0.275,-0.203,-0.2300,-1.38,6,0.54,-0.24,0.13,0.1926,-0.6787,1,0,5,0
2,3,126,10,0.500080,1100,3,83607,1800.0,1527.0,127.76,126.49,7,0.7542,0.7542,1385872,0.7542,0.7542,0.5508,0.7542,0.8091,0.7807,1.308478e+09,89,54,1037651.0,89.0,89.0,100.43,94.37,89.0,0.9175,0,0,0,13,0,0,0.74710,0.75878,0.654685,...,-6.35,14,56.62,-79.48,621.97,1.9592,-0.3632,3.2969,7.2072,5.7062,4.5627,0.226336,3.3277,3.4166,1.8321,0.9979,0.103307,6.8623,5.2963,4.1282,0.219729,8.1381,7.3269,-0.1909,1,-0.58,-0.540,-0.572,-0.3985,-5.18,13,2.89,-1.73,1.04,0.2521,0.7258,1,0,5,0
3,4,134,10,0.439874,1100,3,82642,7542.0,1730.0,132.94,133.58,7,0.8017,0.7881,704687,0.7881,0.7881,0.5923,0.7881,0.8230,0.8158,1.472752e+09,93,55,1115721.0,93.0,93.0,114.63,102.92,93.0,0.9688,0,0,0,4,0,0,0.77405,0.78929,0.750675,...,3.75,5,36.31,18.15,429.81,2.5883,-0.8812,1.2950,2.5777,2.0830,1.6899,0.054630,1.3748,1.3421,0.7982,0.4810,0.081205,2.5571,2.0593,1.6653,0.056470,3.2516,3.0631,-0.1770,1,-0.75,-0.635,-0.745,-0.5100,-2.04,4,1.29,-0.89,0.66,0.2498,0.7119,1,0,5,0
4,5,109,9,0.502749,2900,4,79124,89.0,491.0,122.72,112.77,6,0.5263,0.5263,51985,0.5263,0.5263,0.3044,0.5405,0.5556,0.5455,1.442916e+09,60,21,536400.0,60.0,60.0,60.00,60.00,60.0,0.8451,0,0,0,26,0,0,0.60315,0.53367,0.696150,...,11.45,27,2206.14,-10559.05,708436.49,9.0293,-0.8760,6.3852,17.2599,14.2795,11.9179,0.085330,7.2175,6.2262,3.1446,1.6149,0.074286,15.9080,12.5688,9.9844,0.067540,17.5561,15.6079,-0.4444,1,-0.18,-0.280,-0.182,-0.4277,-11.12,26,6.11,-3.82,2.51,0.2282,-0.5399,0,0,5,0


,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,...,f739,f740,f741,f742,f743,f744,f745,f746,f747,f748,f749,f750,f751,f752,f753,f754,f755,f756,f757,f758,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768,f769,f770,f771,f772,f773,f774,f775,f776,f777,f778
0,105472,147,6,0.487058,1100,17.0,75506,964.0,12686.0,152.63,115.91,12,0.8624,0.7500,25503199,0.9043,0.7016,0.6132,0.7025,0.9048,0.7699,1.751211e+09,77,75,974283.0,134.39,85.00,114.76,96.15,92.0,0.9434,0,0,0,20,0,0,0.55645,0.21888,0.707100,...,1.2006,8.00,9,109.42,19551.94,36228.78,4.8428,0.3641,5.8318,0.9244,5.0663,7.2196,0.336641,4.0067,0.6566,4.5944,0.1807,0.100164,5.9264,1.1442,0.6727,0.081454,6.7791,1.1715,NaN,1,-0.62,-0.310,-0.315,-0.7367,-8.71,19,3.30,-9.37,0.50,0.0539,-1.0733,0,1,1079
1,105473,156,6,0.728518,5400,3.0,79754,1455.0,4803.0,153.95,155.50,12,0.6639,0.6220,1679764,0.6077,0.6077,0.3575,0.6583,0.6991,0.6695,5.286814e+09,79,76,942391.0,121.53,97.35,138.01,103.54,79.0,0.7900,0,0,0,24,0,0,0.69385,0.62693,0.548275,...,0.1640,4.10,25,557.84,1269.30,31543.13,4.7209,0.7232,5.7532,12.0610,10.1518,8.6551,0.365924,6.3457,7.4756,4.4763,2.7723,0.130057,11.0000,8.8048,7.1263,0.347753,14.5358,13.5729,-0.3009,1,-0.31,-0.555,-0.369,-0.5525,-13.26,24,9.53,-7.55,6.22,0.3030,0.6087,0,1,36
2,105474,132,9,0.898133,2200,16.0,113,5735.0,2387.0,130.61,131.96,13,0.7937,0.7937,15984334,0.8197,0.7692,0.6350,0.8621,0.8696,0.8696,3.484800e+09,100,75,1320000.0,100.00,100.00,139.95,117.70,100.0,0.9709,0,0,0,9,0,0,0.79460,0.77228,0.734211,...,1.1870,11.87,10,69.14,240.57,1333.35,2.3463,0.5994,2.8146,6.4241,5.5673,4.8905,0.114978,3.0063,2.8477,1.7155,1.0628,0.132448,6.0645,5.0808,4.3027,0.100486,7.5185,7.1441,-0.1304,1,-0.71,-0.735,-0.719,-0.5544,-4.99,9,3.25,-2.33,1.69,0.2317,0.4184,0,0,393
3,105475,128,7,0.038411,1300,4.0,3793,4689.0,3469.0,120.50,121.93,9,0.6949,0.7500,247163,0.6341,0.5794,0.4848,0.8560,0.8246,0.8333,7.957115e+09,91,124,1516320.0,140.88,91.57,88.75,98.00,90.0,0.8529,0,0,0,3,0,0,0.77905,0.75680,0.763683,...,0.2494,-0.50,3,5092.64,-1060.09,23923.76,4.6887,0.6893,2.2608,10.4271,6.7042,26.9784,0.062658,2.2320,4.9000,6.2552,1.7898,0.198527,27.6734,27.2607,28.7848,0.372096,21.5487,1.2700,-1.0000,1,-0.93,-0.395,-0.080,-0.4017,-16.83,11,0.26,-5.31,0.78,0.2826,-0.7711,0,0,394
4,105476,119,10,0.443620,1300,16.0,13026,2788.0,7438.0,127.00,125.98,12,0.9032,0.0000,6543726,0.0000,0.6724,0.5079,0.7154,0.8661,0.6549,4.993310e+09,72,49,1051687.0,103.00,96.00,99.94,48.26,77.0,0.8587,0,0,0,20,0,0,0.76810,0.75689,0.744667,...,-1.0108,0.78,47,177.32,-313.53,7065.21,6.0078,0.9557,1.5397,3.9892,1.0136,0.5248,0.259062,4.8046,14.8373,1.2512,2.4337,0.102604,14.0360,2.6312,13.9360,0.007415,8.3913,1.8831,-0.0631,1,-0.50,-0.320,-0.337,-0.5488,-20.00,10,13.55,-0.61,0.01,0.1815,-1.0843,0,0,23


In [ ]:
Loan_Data.info()
Loan_Data_Test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 771 entries, id to loss
dtypes: float64(653), int64(99), object(19)
memory usage: 620.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210944 entries, 0 to 210943
Columns: 770 entries, id to f778
dtypes: float64(654), int64(96), object(19), uint64(1)
memory usage: 1.2+ GB


In [ ]:
c=Loan_Data.keys()
ct=Loan_Data_Test.keys()
c
ct

Index(['id', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9',
       ...
       'f770', 'f771', 'f772', 'f773', 'f774', 'f775', 'f776', 'f777', 'f778',
       'loss'],
      dtype='object', length=771)

Index(['id', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9',
       ...
       'f769', 'f770', 'f771', 'f772', 'f773', 'f774', 'f775', 'f776', 'f777',
       'f778'],
      dtype='object', length=770)

In [ ]:
Loan_Data.isnull().sum()
Loan_Data_Test.isnull().sum()

id         0
f1         0
f2         0
f3         0
f4         0
        ... 
f775    1525
f776       0
f777       0
f778       0
loss       0
Length: 771, dtype: int64

id         0
f1         0
f2         0
f3         0
f4         0
        ... 
f774    2066
f775    2960
f776       0
f777       0
f778       0
Length: 770, dtype: int64

In [ ]:
Loan_Data.dropna(inplace=True)
Loan_Data_Test.dropna(inplace=True)

In [ ]:
for i in Loan_Data.select_dtypes(include=['object']).columns:
    Loan_Data.drop(labels=i, axis=1, inplace=True)
for i in Loan_Data_Test.select_dtypes(include=['object']).columns:
    Loan_Data_Test.drop(labels=i, axis=1, inplace=True)

In [ ]:
Loan_Data.shape
Loan_Data_Test.shape

(51940, 752)

(54767, 751)

In [ ]:
Loan_Data.head()
Loan_Data_Test.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,...,f740,f741,f742,f743,f744,f745,f746,f747,f748,f749,f750,f751,f752,f753,f754,f755,f756,f757,f758,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768,f769,f770,f771,f772,f773,f774,f775,f776,f777,f778,loss
2,3,126,10,0.500080,1100,3,83607,1800.0,1527.0,127.76,126.49,7,0.7542,0.7542,1385872,0.7542,0.7542,0.5508,0.7542,0.8091,0.7807,1.308478e+09,89,54,1037651.0,89.0,89.0,100.43,94.37,89.0,0.9175,0,0,0,13,0,0,0.74710,0.75878,0.654685,...,-6.35,14,56.62,-79.48,621.97,1.9592,-0.3632,3.2969,7.2072,5.7062,4.5627,0.226336,3.3277,3.4166,1.8321,0.9979,0.103307,6.8623,5.2963,4.1282,0.219729,8.1381,7.3269,-0.1909,1,-0.58,-0.540,-0.572,-0.3985,-5.18,13,2.89,-1.73,1.04,0.2521,0.7258,1,0,5,0
4,5,109,9,0.502749,2900,4,79124,89.0,491.0,122.72,112.77,6,0.5263,0.5263,51985,0.5263,0.5263,0.3044,0.5405,0.5556,0.5455,1.442916e+09,60,21,536400.0,60.0,60.0,60.00,60.00,60.0,0.8451,0,0,0,26,0,0,0.60315,0.53367,0.696150,...,11.45,27,2206.14,-10559.05,708436.49,9.0293,-0.8760,6.3852,17.2599,14.2795,11.9179,0.085330,7.2175,6.2262,3.1446,1.6149,0.074286,15.9080,12.5688,9.9844,0.067540,17.5561,15.6079,-0.4444,1,-0.18,-0.280,-0.182,-0.4277,-11.12,26,6.11,-3.82,2.51,0.2282,-0.5399,0,0,5,0
6,7,121,9,0.985674,2900,4,13026,4565.0,263.0,126.36,122.09,6,0.7739,0.7739,542244,0.7739,0.7417,0.5378,0.8091,0.8165,0.8165,3.193177e+09,89,64,1045572.0,89.0,89.0,89.00,89.00,89.0,0.9674,0,0,0,23,0,0,0.79085,0.74753,0.771352,...,39.74,24,565.50,5004.95,71164.89,4.5630,1.0192,8.4532,17.8793,15.9062,14.2316,0.069310,9.1465,7.9037,4.7938,2.9707,0.088964,16.8881,14.5864,12.6635,0.062523,19.4102,18.4200,-0.1835,1,-0.67,-0.605,-0.653,-0.4965,-11.42,23,7.06,-4.99,3.77,0.2458,-0.6732,0,0,5,0
7,8,128,9,0.385778,2900,4,79244,6597.0,3592.0,127.19,127.89,6,0.8596,0.7967,17175731,0.8033,0.7538,0.5868,0.8376,0.9159,0.8522,3.127331e+09,98,75,1219708.0,98.0,98.0,120.79,108.46,98.0,0.9899,0,0,0,17,0,0,0.72690,0.73842,0.739165,...,21.49,18,177.11,361.87,5280.10,2.9007,-0.7834,5.5179,11.5597,9.6605,8.1316,0.082056,6.0522,5.1945,3.0525,1.8459,0.120041,10.7886,8.7296,7.1231,0.084859,13.8244,13.0117,-0.0841,1,-0.71,-0.475,-0.649,-0.4182,-7.11,17,4.45,-3.26,2.56,0.2947,-0.7220,0,0,5,1
8,9,126,9,0.745471,2900,4,78920,3058.0,112.0,123.89,125.53,6,0.8684,0.8049,1560191,0.8115,0.7615,0.5705,0.8462,0.9252,0.8609,3.040701e+09,99,72,1185921.0,99.0,99.0,122.03,109.56,99.0,1.0000,0,0,0,7,0,0,0.76075,0.76535,0.657429,...,9.74,8,45.34,98.11,422.89,2.0458,-0.8261,2.1964,4.5327,3.9902,3.5362,0.312681,2.5327,2.2290,1.3060,0.7808,0.093893,4.2704,3.6432,3.1261,0.302410,5.2912,5.0922,-0.0748,1,-0.86,-0.660,-0.814,-0.4700,-3.29,7,2.02,-1.35,0.95,0.2601,0.7132,0,0,5,0


,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,...,f739,f740,f741,f742,f743,f744,f745,f746,f747,f748,f749,f750,f751,f752,f753,f754,f755,f756,f757,f758,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768,f769,f770,f771,f772,f773,f774,f775,f776,f777,f778
2,105474,132,9,0.898133,2200,16.0,113,5735.0,2387.0,130.61,131.96,13,0.7937,0.7937,15984334,0.8197,0.7692,0.6350,0.8621,0.8696,0.8696,3.484800e+09,100,75,1320000.0,100.00,100.00,139.95,117.70,100.0,0.9709,0,0,0,9,0,0,0.79460,0.77228,0.734211,...,1.1870,11.87,10,69.14,240.57,1333.35,2.3463,0.5994,2.8146,6.4241,5.5673,4.8905,0.114978,3.0063,2.8477,1.7155,1.0628,0.132448,6.0645,5.0808,4.3027,0.100486,7.5185,7.1441,-0.1304,1,-0.71,-0.735,-0.719,-0.5544,-4.99,9,3.25,-2.33,1.69,0.2317,0.4184,0,0,393
9,105481,153,7,0.445673,5300,10.0,83293,4121.0,455.0,154.38,153.20,9,0.9204,0.8966,3624718,0.8667,0.8667,0.7572,0.9204,0.9369,0.9286,8.114163e+09,104,112,1676480.0,168.41,131.38,171.32,132.48,104.0,0.9905,0,0,0,34,0,0,0.83335,0.86003,0.571682,...,0.1006,3.52,35,1359.61,-2824.45,129980.31,6.2318,-0.7255,9.0473,19.0198,16.2105,13.9439,0.346304,10.8121,11.2510,7.1768,4.7455,0.182069,18.1277,15.0993,12.6971,0.338628,23.6960,22.3620,-0.0631,1,-0.78,-0.730,-0.772,-0.5441,-18.50,34,12.15,-8.58,6.26,0.2476,0.8338,0,1,34
12,105484,127,8,0.870942,1300,4.0,84020,4606.0,9204.0,125.60,126.84,9,0.8291,0.7886,5227446,0.7886,0.7823,0.6261,0.8083,0.8739,0.8509,1.825722e+09,97,66,1185534.0,97.00,97.00,119.56,107.35,97.0,0.9238,0,0,0,17,0,0,0.79115,0.78407,0.783324,...,-0.2122,-3.82,18,381.54,-2062.59,29924.08,4.5991,-1.0130,6.5216,12.1948,10.3647,8.8288,0.041289,6.8774,6.1078,3.7409,2.3232,0.072508,11.7679,9.8277,8.2264,0.041558,14.5181,13.8168,-0.1261,1,-0.31,-0.315,-0.309,-0.4506,-7.66,17,4.61,-3.20,2.43,0.2610,-0.6390,0,0,30
15,105487,125,10,0.153149,1300,16.0,84882,1715.0,2377.0,126.12,125.22,11,0.8421,0.8067,1026989,0.7805,0.7805,0.6064,0.8205,0.8496,0.8496,1.310884e+09,96,51,1170432.0,96.00,96.00,96.00,96.00,96.0,0.9231,0,0,0,22,0,0,0.76025,0.78245,0.741227,...,1.6061,36.94,23,500.32,2354.26,27209.19,4.3788,0.4111,7.0341,14.3973,11.8165,9.7774,0.082375,7.1969,6.2383,3.4794,1.9906,0.097381,13.7121,10.9821,8.8678,0.079480,17.0244,15.8333,-0.1504,1,-0.38,-0.480,-0.412,-0.5291,-11.64,22,7.94,-5.73,4.21,0.2846,0.8886,0,0,393
18,105490,112,10,0.044429,1900,16.0,76878,479.0,6496.0,117.08,112.70,9,0.7478,0.7049,158450,0.6880,0.6880,0.4433,0.7288,0.7748,0.7748,1.920002e+09,86,52,872728.0,86.00,86.00,120.36,101.22,86.0,0.9556,0,0,0,13,0,0,0.70400,0.68920,0.710823,...,-0.3850,-5.39,14,468.69,-1196.01,30833.48,5.7732,-0.6246,3.4758,7.8935,6.2182,4.9280,0.069636,3.6842,3.1763,1.6217,0.8399,0.079274,7.7233,6.0079,4.6967,0.063837,10.2930,9.6451,-0.2252,1,-0.67,-0.625,-0.663,-0.4854,-6.31,13,4.26,-3.15,2.44,0.3035,0.6035,1,0,513


In [ ]:
y=Loan_Data.pop('loss'),
y[:3]
X=Loan_Data,
X= np.asarray(X)
y= np.asarray(y)

(2         0
 4         0
 6         0
 7         1
 8         0
          ..
 105463    3
 105464    0
 105465    0
 105466    0
 105468    0
 Name: loss, Length: 51940, dtype: int64,)

In [ ]:
X.shape
y.shape

(1, 51940, 751)

(1, 51940)

In [ ]:
X= X.reshape(-1,751)
y= y.reshape(51940)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [ ]:
f"X_train shape: {X_train.shape}" 
print()
f"X_test.shape : {X_test.shape}"  
print()
f"y_train shape: {y_train.shape}" 
print()
f"y_test shape : {y_test.shape}" 

'X_train shape: (41552, 751)'

'X_test.shape : (10388, 751)'

'y_train shape: (41552,)'

'y_test shape : (10388,)'

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred= model.predict(X_test)
print(accuracy_score( y_test, y_pred))

0.9003658067000385


In [ ]:
X_test1= Loan_Data_Test
y_pred1= model.predict(X_test1)

In [ ]:
print(y_pred1)

[0 0 0 ... 0 0 0]
